In [9]:
import numpy as np

In [59]:
moves = ["G1","G2","G3"]

def extract_coords_2d(cmd):
    words = cmd.split(" ")
    x,y = None, None
    for w in words:
        if words[0] in moves:
            if "X" in w:
                x = float(w[1:])
            if "Y" in w:
                y = float(w[1:])
    return (x,y)

def extract_rotation_centers(cmd):
    words = cmd.split(" ")
    i,j = 0, 0
    for w in words:
        if words[0] in moves:
            if "I" in w:
                i = float(w[1:])
            if "J" in w:
                j = float(w[1:])
    return (i,j)

In [31]:
f = open("gcode/tube_path_explicit.pgm","r")
lines = f.read().splitlines()
f.close()

mvt_lines = []
commands = []
coords = []
rot_centers = []
for l in lines:
    cmd = l.split(" ")[0]
    if cmd not in moves:
        continue
    commands.append(cmd)
    mvt_lines.append(l)
    x,y = extract_coords_2d(l)
    if x == None and y != None:
        x = coords[-1][0]
    if x != None and y == None:
        y = coords[-1][1]
    coords.append([x,y])
    i,j = extract_rotation_centers(l)
    rot_centers.append([i,j])

In [47]:
output = ["G91","ENABLE X Y","G16 X Y Z","G17"]
for k in range(1,len(mvt_lines)):
    l = mvt_lines[k]
    cmd = commands[k]
    dx = coords[k][0] - coords[k-1][0]
    dy = coords[k][1] - coords[k-1][1]
    i,j = rot_centers[k][0],rot_centers[k][1]
    if cmd == "G1":
        line = "G1 X{0:.4f} Y{1:.4f} F40".format(dx,dy)
    if cmd == "G2" or cmd == "G3":
        line = "{0} X{1:.4f} Y{2:.4f} I{3:.4f} J{4:.4f}".format(cmd,dx,dy,i,j)
    output.append(line)
output.append("DISABLE X Y")

In [51]:
fout = open("gcode/tube_path_relative.pgm","w")
for l in output:
    fout.write(l+"\n")
fout.close()

In [60]:
rot_deg = 90
rot = rot_deg*np.pi/180
mtx = np.array([[np.cos(rot),-np.sin(rot)],[np.sin(rot),np.cos(rot)]])
coords_rot = [np.dot(mtx,vec) for vec in coords]
rot_centers_rot = [np.dot(mtx,vec) for vec in rot_centers]
output_rot = ["G91","ENABLE X Y","G16 X Y Z","G17"]
for k in range(1,len(mvt_lines)):
    l = mvt_lines[k]
    cmd = commands[k]
    dx = coords_rot[k][0] - coords_rot[k-1][0]
    dy = coords_rot[k][1] - coords_rot[k-1][1]
    i,j = rot_centers_rot[k][0],rot_centers_rot[k][1]
    if cmd == "G1":
        line = "G1 X{0:.4f} Y{1:.4f} F40".format(dx,dy)
    if cmd == "G2" or cmd == "G3":
        line = "{0} X{1:.4f} Y{2:.4f} I{3:.4f} J{4:.4f}".format(cmd,dx,dy,i,j)
    output_rot.append(line)
output_rot.append("DISABLE X Y")

NameError: name 'rot_centers' is not defined

In [57]:
fout = open("gcode/tube_path_relative_rotate{0}.pgm".format(rot_deg),"w")
for l in output_rot:
    fout.write(l+"\n")
fout.close()